In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Apprentissage de Pyspark par la pratique") \
    .getOrCreate()

22/11/12 21:21:41 WARN Utils: Your hostname, cniongolo.local resolves to a loopback address: 127.0.0.1; using 192.168.1.182 instead (on interface en0)
22/11/12 21:21:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/12 21:21:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
spark.version

'3.3.1'

In [ ]:
conso_elec_gaz_annuelle = spark.read.option("delimiter", ";").csv("datasets/conso-elec-gaz-annuelle-par-secteur-dactivite-agregee-departement.csv", header=True, inferSchema=True)

In [19]:
conso_elec_gaz_annuelle.take(5)

[Row(operateur='Enedis', annee=2011, filiere='Electricité', consoa=28496.17, pdla=280, nombre_maille_secretisees_a=0, indquala=0.943, consoi=1180989.65, pdli=1412, nombre_maille_secretisees_i=0, indquali=0.964, consot=737572.47, pdlt=3040, nombre_maille_secretisees_t=0, indqualt=0.932, consor=1977574.54, pdlr=313211, nombre_maille_secretisees_r=0, indqualr=0.303, consona=37947.56, pdlna=39, nombre_maille_secretisees_na=0, indqualna=0.829, code_departement='01', libelle_departement='Ain', code_region=84, libelle_region='Auvergne-Rhône-Alpes', id_filiere=100, geom=None, consototale=3962580.39),
 Row(operateur='Enedis', annee=2011, filiere='Electricité', consoa=5096.35, pdla=66, nombre_maille_secretisees_a=0, indquala=0.86, consoi=348627.93, pdli=893, nombre_maille_secretisees_i=0, indquali=0.903, consot=2104796.95, pdlt=8006, nombre_maille_secretisees_t=0, indqualt=0.923, consor=4113958.18, pdlr=823968, nombre_maille_secretisees_r=1, indqualr=0.354, consona=12681.93, pdlna=134, nombre_ma